In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable as V
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
batch_size = 64

train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [3]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [4]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [7]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = V(data), V(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = V(data, volatile=True), V(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
for epoch in range(1, 10):
    train(epoch)
    test()

/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313707
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.300028
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.293394
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.296297
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.298548
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.293348
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.292526
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.273361
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.286131
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.295617
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.274274
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.261155
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.240649
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.216738
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.148423
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.058023
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.915219
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.690762
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.558148
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.123118
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.124327
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.358623
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.152492
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.168262
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.028262
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.147210
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.200457
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.057366
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.030354
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.069897
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.134622
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.170053
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.118697
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.120369
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.161240
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.174930
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.100396
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.168138
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.193620
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.129032


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.030772
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.102350
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.021451
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.052166
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.077535
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.081193
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.030236
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.076324
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.172777
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.125141
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.164659
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.159193
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.011569
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.044769
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.121710
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.200364
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.107582
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.143675
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.075383
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.038089


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.047742
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.070939
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.089592
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.018249
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.008519
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.019033
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.034552
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.063765
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.041526
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.218198
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.078511
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.005478
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.054810
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.085234
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.098308
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.029178
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.119159
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.100690
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.057523
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.092704


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.033398
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.033081
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.073086
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.037565

Test set: Average loss: 0.0494, Accuracy: 9835/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.010539
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.346256
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.009406
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.016772
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.040492
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.064543
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.011818
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.081244
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.007782
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.047878
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.030969
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.023165
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.017124
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.010091
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.055699
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.021159
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.008369
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.012431
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.015345
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.152096
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.112739
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.014994
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.102446
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.050082
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.015411
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.012355
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.007530
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.026352
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.038164
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.040297
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.015371
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.038762
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.069586
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.016670
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.017725
